In [2]:
import re

## Input examples

In [3]:
code = "(first (list 1 (+ 2 3) (- 5 4) (* 2 1) (/ 6 3) 9))"
code_bad = "(first (list 1 (+2 3) (- 5 4) (* 2 1) (/ 6 3) 9))"

## Lexer

### Using regex

Why beceause I read the note[1] at the end of the `split()` documentation too
quick.

[1]: https://koor.fr/Python/API/python/builtins/str/split.wp (that's what
    appears when I hover over `split`)

In [4]:
tokens = [token.strip() for token in re.split(r'(\(|\)|\s)', code)]
tokens = [token for token in tokens if token != '']
tokens

['(',
 'first',
 '(',
 'list',
 '1',
 '(',
 '+',
 '2',
 '3',
 ')',
 '(',
 '-',
 '5',
 '4',
 ')',
 '(',
 '*',
 '2',
 '1',
 ')',
 '(',
 '/',
 '6',
 '3',
 ')',
 '9',
 ')',
 ')']

### Bad idea trying to create the string representation of a Python list and then using `eval()` to get the list back

In [5]:
code_list = code.replace('(', '[').replace(')', ']').replace(' ', ', ')

In [6]:
code_list.strip('][').split(', ')

['first',
 '[list',
 '1',
 '[+',
 '2',
 '3]',
 '[-',
 '5',
 '4]',
 '[*',
 '2',
 '1]',
 '[/',
 '6',
 '3]',
 '9']

In [7]:
eval(code_list)

SyntaxError: invalid syntax (<string>, line 1)

### A much simpler way, using `replace()` and `split()`

In [8]:
code.replace('(', ' ( ').replace(')', ' ) ').split()

['(',
 'first',
 '(',
 'list',
 '1',
 '(',
 '+',
 '2',
 '3',
 ')',
 '(',
 '-',
 '5',
 '4',
 ')',
 '(',
 '*',
 '2',
 '1',
 ')',
 '(',
 '/',
 '6',
 '3',
 ')',
 '9',
 ')',
 ')']

In [33]:
def tokenize(source_code):
  return source_code.replace('(', ' ( ').replace(')', ' ) ').split()

## Parser

For now, my tiny_list will only contain procedures (treated as strings) and numbers
(positive integers only).

EXPR = ATOM | '(' atom expression ')'

### Recursive

In [9]:
def parse(tokens, index=0):

    expression = []

    while index < len(tokens):
        token = tokens[index]

        if token == ')':
            return expression, index
        elif token == '(':
            sub_expression, index = parse(tokens, index + 1)
            expression.append(sub_expression)
        elif token.isdigit():
            expression.append(int(token))
        else:
            expression.append(token)
        
        index += 1
    
    return expression


In [10]:
parse(tokens)

[['first', ['list', 1, ['+', 2, 3], ['-', 5, 4], ['*', 2, 1], ['/', 6, 3], 9]]]

While is probably missplaced.

In [ ]:
class ParseError(ValueError):
    pass

In [119]:
def _parse(tokens, index=0):

    token = tokens[index]

    if token == ")":
        raise ParseError("Unexpected `)`")
    elif token == "(":
        expression = []

        index += 1
        while index < len(tokens) and tokens[index] != ")":
            sub_expression, index = _parse(tokens, index)
            expression.append(sub_expression)

        if index == len(tokens):
            raise ParseError(
                "Unexpected EOF. A closing parentheses `)` might be missing."
            )

        return expression, index + 1
    elif token.isdigit():
        return int(token), index + 1
    else:
        return token, index + 1


def parse(tokens):

    # Empty input
    if len(tokens) == 0:
        return []

    ast, index = _parse(tokens, index=0)

    if index != len(tokens):
        raise ParseError("You have unreachable code.")

    return ast


This should be better.

### Tests

In [120]:
print(code)
parse(tokenize(code))

(first (list 1 (+ 2 3) (- 5 4) (* 2 1) (/ 6 3) 9))


['first', ['list', 1, ['+', 2, 3], ['-', 5, 4], ['*', 2, 1], ['/', 6, 3], 9]]

In [121]:
parse(tokenize(""))

[]

In [122]:
parse(tokenize("(+ 1 2"))

NameError: name 'ParseError' is not defined

In [123]:
parse(tokenize(")"))

NameError: name 'ParseError' is not defined

In [124]:
parse(tokenize("(+ 1 2) 3"))

NameError: name 'ParseError' is not defined

## Evaluation

In [125]:
class EvalError(ValueError):
    pass

In [211]:
def add(args):
    sum = 0

    for arg in args:
        arg = eval(arg)
        
        if type(arg) != int:
            raise TypeError("Args of `+` should evaluate to `int`.")
        
        sum += arg

    return sum

In [198]:
def substract(args):

    if args == []:
        raise ValueError("`-` takes one or more term.")

    first_value = args.pop(0)
    first_value = eval(first_value)

    if type(first_value) != int:
        raise TypeError("Args of `-` should evaluate to `int`.")

    if args == []:
        return - first_value
    
    difference = first_value


    for arg in args:
        arg = eval(arg)
        
        if type(arg) != int:
            raise TypeError("Args of `-` should evaluate to `int`.")
        
        difference -= arg

    return difference

In [216]:
def multiply(args):
    product = 1

    for arg in args:
        arg = eval(arg)
        
        if type(arg) != int:
            raise TypeError("Args of `*` should evaluate to `int`.")
        
        product *= arg

    return product

In [231]:
def eval(ast):
    # Atom: integer
    if type(ast) is int:
        return ast

    if type(ast) is not list:
        raise EvalError(
            "This interpreter only accepts integers and operations, for now."
        )
    
    operation = ast.pop(0)

    match operation:
        case "+":
            return add(ast)
        case "-":
            return substract(ast)
        case "*":
            return multiply(ast)

    raise EvalError("Unexpected error. There are probably still bits to implement.")


### Tests


In [219]:
# Simple integer
eval(parse(tokenize("42")))

42

In [220]:
# Integer call
eval(parse(tokenize("(42)")))

EvalError: Unexpected error. There are probably still bits to implement.

In [221]:
# String (unrecognized symbol for now)
eval(parse(tokenize("Hello")))

EvalError: This interpreter only accepts integers and operations, for now.

In [222]:
# Simple addition, 2 terms
eval(parse(tokenize("(+ 23 19)")))

42

In [223]:
# Scheme addition, many terms
eval(
    parse(
        tokenize(
            "(+ 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1)"
        )
    )
)


42

In [238]:
# Scheme addition, 1 term
eval(parse(tokenize("(+ 42)")))

42

In [237]:
# Scheme addition, no term.
eval(parse(tokenize("(+)")))
# 0

0

In [226]:
# Simple substraction, 2 terms
eval(parse(tokenize("(- 51 9)")))

42

In [227]:
# Scheme substraction, many terms
eval(parse(tokenize("(- 51 1 1 1 1 1 1 1 1 1)")))

42

In [228]:
# Scheme substraction, 1 term
eval(parse(tokenize("(- 42)")))
# -42

-42

In [229]:
# Scheme substraction, no term
eval(parse(tokenize("(-)")))
# Error

ValueError: `substract` takes one or more term.

In [232]:
# Simple multiplication, 2 terms
eval(parse(tokenize("(* 21 2)")))

42

In [234]:
# Scheme multiplication, many terms
eval(
    parse(
        tokenize(
            "(* 3 7 2)"
        )
    )
)

42

In [235]:
# Scheme addition, 1 term
eval(parse(tokenize("(* 42)")))

42

In [236]:
# Scheme mutliplication, no term.
eval(parse(tokenize("(*)")))
# 1

1